<a href="https://colab.research.google.com/github/harunpirim/IME775/blob/main/week-04/notebook-math-dl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>---

# Week 4: Neural Network Foundations - Perceptrons to MLPs
**IME775: Data Driven Modeling and Optimization**
📖 **Reference**: Krishnendu Chaudhury. *Math and Architectures of Deep Learning*, Chapter 5
---
## Learning Objectives
- Understand the perceptron as a linear classifier
- Master multi-layer network mathematics
- Learn forward propagation computation
- Visualize decision boundaries


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

## 4.1 The Perceptron: A Single Neuron
$$y = \sigma(w^T x + b) = \sigma\left(\sum_{i=1}^n w_i x_i + b\right)$$
The perceptron defines a **hyperplane** that separates classes.


In [ ]:
# Visualize single perceptron decision boundary
fig1, axes1 = plt.subplots(1, 2, figsize=(14, 5))
# Generate linearly separable data
np.random.seed(42)
n_samples = 100
X_class0 = np.random.randn(n_samples // 2, 2) + np.array([2, 2])
X_class1 = np.random.randn(n_samples // 2, 2) + np.array([-2, -2])
X_linear = np.vstack([X_class0, X_class1])
y_linear = np.array([0] * (n_samples // 2) + [1] * (n_samples // 2))
# Define perceptron weights (learned or set manually)
w = np.array([1.0, 1.0])
b = 0.0
# Decision boundary: w^T x + b = 0  =>  x2 = -(w1*x1 + b)/w2
x1_line = np.linspace(-5, 5, 100)
x2_line = -(w[0] * x1_line + b) / w[1]
# Plot
ax1 = axes1[0]
ax1.scatter(X_class0[:, 0], X_class0[:, 1], c='blue', label='Class 0', alpha=0.6)
ax1.scatter(X_class1[:, 0], X_class1[:, 1], c='red', label='Class 1', alpha=0.6)
ax1.plot(x1_line, x2_line, 'k-', linewidth=2, label='Decision boundary')
# Add weight vector
ax1.quiver(0, 0, w[0], w[1], angles='xy', scale_units='xy', scale=1, 
          color='green', width=0.03, label='Weight vector')
ax1.axhline(0, color='gray', linewidth=0.5)
ax1.axvline(0, color='gray', linewidth=0.5)
ax1.set_xlim(-5, 5)
ax1.set_ylim(-5, 5)
ax1.set_xlabel('x₁')
ax1.set_ylabel('x₂')
ax1.set_title('Perceptron: Linear Decision Boundary')
ax1.legend()
ax1.grid(True, alpha=0.3)
# XOR problem - not linearly separable
ax2 = axes1[1]
X_xor = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y_xor = np.array([0, 1, 1, 0])
colors_xor = ['blue' if y == 0 else 'red' for y in y_xor]
ax2.scatter(X_xor[:, 0], X_xor[:, 1], c=colors_xor, s=200, edgecolors='black')
for i, (x, y) in enumerate(zip(X_xor, y_xor)):
    ax2.annotate(f'XOR={y}', xy=(x[0], x[1]), xytext=(x[0]+0.1, x[1]+0.1), fontsize=12)
ax2.set_xlim(-0.5, 1.5)
ax2.set_ylim(-0.5, 1.5)
ax2.set_xlabel('x₁')
ax2.set_ylabel('x₂')
ax2.set_title('XOR Problem: NOT Linearly Separable\n(Need multiple layers!)')
ax2.grid(True, alpha=0.3)
plt.tight_layout()
fig1

## 4.2 Activation Functions
| Function | Formula | Use Case |
|----------|---------|----------|
| Sigmoid | $\frac{1}{1+e^{-x}}$ | Output for binary classification |
| ReLU | $\max(0, x)$ | Hidden layers (most common) |
| Tanh | $\frac{e^x - e^{-x}}{e^x + e^{-x}}$ | Hidden layers (zero-centered) |
| Softmax | $\frac{e^{x_i}}{\sum_j e^{x_j}}$ | Multi-class output |


In [ ]:
# Visualize activation functions
fig2, axes2 = plt.subplots(2, 2, figsize=(14, 10))
x_act = np.linspace(-5, 5, 200)
# Sigmoid
sigmoid = 1 / (1 + np.exp(-x_act))
axes2[0, 0].plot(x_act, sigmoid, 'b-', linewidth=2, label='σ(x)')
axes2[0, 0].axhline(0.5, color='gray', linestyle='--', alpha=0.5)
axes2[0, 0].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes2[0, 0].fill_between(x_act, sigmoid, alpha=0.2)
axes2[0, 0].set_title('Sigmoid: Output ∈ (0, 1)')
axes2[0, 0].set_xlabel('x')
axes2[0, 0].set_ylabel('σ(x)')
axes2[0, 0].legend()
axes2[0, 0].grid(True, alpha=0.3)
# Tanh
tanh = np.tanh(x_act)
axes2[0, 1].plot(x_act, tanh, 'g-', linewidth=2, label='tanh(x)')
axes2[0, 1].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes2[0, 1].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes2[0, 1].fill_between(x_act, tanh, alpha=0.2, color='green')
axes2[0, 1].set_title('Tanh: Output ∈ (-1, 1), Zero-Centered')
axes2[0, 1].set_xlabel('x')
axes2[0, 1].set_ylabel('tanh(x)')
axes2[0, 1].legend()
axes2[0, 1].grid(True, alpha=0.3)
# ReLU
relu = np.maximum(0, x_act)
axes2[1, 0].plot(x_act, relu, 'r-', linewidth=2, label='ReLU(x)')
axes2[1, 0].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes2[1, 0].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes2[1, 0].fill_between(x_act, relu, alpha=0.2, color='red')
axes2[1, 0].set_title('ReLU: Non-Saturating for x > 0')
axes2[1, 0].set_xlabel('x')
axes2[1, 0].set_ylabel('ReLU(x)')
axes2[1, 0].legend()
axes2[1, 0].grid(True, alpha=0.3)
# Leaky ReLU and variants
leaky_relu = np.where(x_act > 0, x_act, 0.1 * x_act)
elu = np.where(x_act > 0, x_act, np.exp(x_act) - 1)
axes2[1, 1].plot(x_act, relu, 'r-', linewidth=2, label='ReLU', alpha=0.5)
axes2[1, 1].plot(x_act, leaky_relu, 'm-', linewidth=2, label='Leaky ReLU (α=0.1)')
axes2[1, 1].plot(x_act, elu, 'c-', linewidth=2, label='ELU')
axes2[1, 1].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes2[1, 1].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes2[1, 1].set_title('ReLU Variants: Avoid Dead Neurons')
axes2[1, 1].set_xlabel('x')
axes2[1, 1].set_ylabel('f(x)')
axes2[1, 1].legend()
axes2[1, 1].grid(True, alpha=0.3)
plt.tight_layout()
fig2

## 4.3 Multi-Layer Perceptron (MLP)
**Forward Propagation**:
For layer $l$:
$$z^{(l)} = W^{(l)} h^{(l-1)} + b^{(l)}$$
$$h^{(l)} = \sigma(z^{(l)})$$
Stacking layers enables learning complex non-linear functions!


In [ ]:
# Implement and visualize MLP solving XOR
class SimpleMLP:
    def __init__(self, layer_sizes):
        np.random.seed(42)
        self.weights = []
        self.biases = []
        for i in range(len(layer_sizes) - 1):
            W = np.random.randn(layer_sizes[i], layer_sizes[i+1]) * 0.5
            b = np.zeros(layer_sizes[i+1])
            self.weights.append(W)
            self.biases.append(b)
    def forward(self, X):
        self.activations = [X]
        h = X
        for i, (W, b) in enumerate(zip(self.weights, self.biases)):
            z = h @ W + b
            if i < len(self.weights) - 1:
                h = np.maximum(0, z)  # ReLU
            else:
                h = 1 / (1 + np.exp(-z))  # Sigmoid output
            self.activations.append(h)

## 4.4 Universal Approximation
**Theorem**: A single hidden layer network with sufficient neurons can approximate any continuous function.
Let's visualize how more neurons improve approximation!


In [ ]:
# Demonstrate universal approximation
def target_function(x):

## 4.5 Network Architecture Visualization
Understanding how dimensions flow through a network is crucial!


In [ ]:
# Visualize network architecture
def draw_network(ax, layer_sizes, title):
    n_layers = len(layer_sizes)
    max_neurons = max(layer_sizes)
    layer_positions = np.linspace(0, 1, n_layers)
    for i, (pos, n_neurons) in enumerate(zip(layer_positions, layer_sizes)):
        # Calculate vertical positions for neurons
        neuron_positions = np.linspace(0.1, 0.9, min(n_neurons, 10))
        for j, y_pos in enumerate(neuron_positions):
            circle = plt.Circle((pos, y_pos), 0.03, fill=True, 
                               color='lightblue' if i == 0 else ('lightgreen' if i == n_layers-1 else 'lightyellow'),
                               edgecolor='black', linewidth=1)
            ax.add_patch(circle)
        # Draw connections to next layer
        if i < n_layers - 1:
            next_positions = np.linspace(0.1, 0.9, min(layer_sizes[i+1], 10))
            for y1 in neuron_positions[:min(5, len(neuron_positions))]:
                for y2 in next_positions[:min(5, len(next_positions))]:
                    ax.plot([pos, layer_positions[i+1]], [y1, y2], 
                           'gray', linewidth=0.3, alpha=0.3)
        # Add label
        ax.text(pos, -0.05, f'Layer {i}\n({layer_sizes[i]})', ha='center', fontsize=10)
    ax.set_xlim(-0.1, 1.1)
    ax.set_ylim(-0.15, 1.0)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=12, pad=10)
fig5, axes5 = plt.subplots(1, 2, figsize=(14, 6))
# Architecture 1: Wide and shallow
draw_network(axes5[0], [784, 512, 10], 'Shallow: 784 → 512 → 10\nParams: ~400K')
# Architecture 2: Deep and narrow  
draw_network(axes5[1], [784, 256, 128, 64, 10], 'Deep: 784 → 256 → 128 → 64 → 10\nParams: ~240K')
plt.tight_layout()
fig5

In [ ]:
# Parameter counting
def count_params(layer_sizes):
    total = 0
    for i in range(len(layer_sizes) - 1):
        weights = layer_sizes[i] * layer_sizes[i+1]
        biases = layer_sizes[i+1]
        total += weights + biases

## Summary
| Concept | Key Insight |
|---------|-------------|
| **Perceptron** | Linear classifier, limited to linearly separable data |
| **Activation** | Non-linearity enables complex function learning |
| **MLP** | Stacked layers learn hierarchical representations |
| **Universal Approximation** | Sufficient neurons can approximate any function |
---
## References
- **Primary**: Krishnendu Chaudhury. *Math and Architectures of Deep Learning*, Chapter 5.
- **Supplementary**: Goodfellow, I., et al. (2016). *Deep Learning*, Chapter 6.
## Connection to ML Refined Curriculum
Neural networks extend the linear models from Weeks 4-7 to handle non-linear patterns:
- Linear regression → Neural network regression
- Logistic regression → Neural network classification
